In [10]:
import hashlib
import binascii

In [11]:
WORD_SIZE = 0x100000000  #2^32 => 32 bits

In [12]:
# Perform "Circular Rotation" of bit {x} positions
rotate_by_8 = lambda x: ((x << 8) & 0xFFFFFFFF) | (x >> 24)
rotate_by_16 = lambda x: ((x << 16) & 0xFFFFFFFF) | (x >> 16)

In [13]:
# Encide large number n into byte sequence
def encode_large_number(n):
    encoded = ""
    while n > 0:
      # Using bitwise And oparator
      # n & 0xFF (256) => extract the lower 8 bits (1 byte)
      # convert to character using chr
      # Exmaple:
      ### input: 12345678901234567890
      ### output: '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\xb6\xbb4\x98Iv\xd2'
        encoded = chr(n & 0xFF) + encoded
        n >>= 8
    return encoded

In [14]:
# Use square and Xor oparatior to create non-linearily result thus
# Amplify the confusion and diffusion of each value
# The value get module to make sure it stays in the 32 bit length
def non_linear_state_transition(u, v):
    '''Internal non-linear state transition function.'''
    s = (u + v) % WORD_SIZE
    s = s * s
    return (s ^ (s >> 32)) % WORD_SIZE

In [15]:
# Define Rabbit Class
class Rabbit:

    def __init__(self, key, iv=None):
        # Initialize:
        ## 128-bit key(string)
        ## Optional IV

        # Simplify version with key as string
        if len(key) < 16:
            # Padding null byte to ensure 16 char long
            key = '\x00' * (16 - len(key)) + key

        k = []
        # Iterate over the key string in pairs, starting from the 15th character
        for i in range(14, -1, -2):
            # Extract two characters from the string
            char1 = key[i]
            char2 = key[i + 1]
            # Convert characters to ASCII values
            ord_char1 = ord(char1)
            ord_char2 = ord(char2)
            # Combine characters into a 16-bit integer in big-endian byte order
            combined_integer = ord_char2 | (ord_char1 << 8)
            k.append(combined_integer)

        '''Init State and counter'''
        # The key is split into 8 words, each represented as a 16-bit integer in big-endian byte order
        x = []
        for j in range(8):
            if j % 2 == 1:
                value = (k[(j + 5) % 8] << 16) | k[(j + 4) % 8]
            else:
                value = (k[(j + 1) % 8] << 16) | k[j]
            x.append(value)

        c = []
        for j in range(8):
            if j % 2 == 1:
                value = (k[j] << 16) | k[(j + 1) % 8]
            else:
                value = (k[(j + 4) % 8] << 16) | k[(j + 5) % 8]
            c.append(value)

        self.x = x            # internal state
        self.c = c            # internal counter
        self.b = 0            # buffer
        self._buf = 0         # buffer to store the current byte
        self._buf_bytes = 0   # number of bits left in buffer

        # The next(self) method represents the state transition,
        # Calling it multiple times helps in achieving a more complex internal state.
        next(self)
        next(self)
        next(self)
        next(self)

        for j in range(8):
            c[j] ^= x[(j + 4) % 8]

        self.start_x = self.x[:]
        self.start_c = self.c[:]
        self.start_b = self.b

        if iv is not None:
            self.set_iv(iv)

    def set_iv(self, iv):
        # Convert IV to number if provied string
        if isinstance(iv, str):
            i = 0
            for c in iv:
                i = (i << 8) | ord(c)
            iv = i

        c = self.c
        # Break down the 64-bit IV into four 16-bit components
        i0 = iv & 0xFFFFFFFF
        i2 = iv >> 32
        i1 = ((i0 >> 16) | (i2 & 0xFFFF0000)) % WORD_SIZE
        i3 = ((i2 << 16) | (i0 & 0x0000FFFF)) % WORD_SIZE

        # XOR with internal counter to prevent repetition in key stream
        c[0] ^= i0
        c[1] ^= i1
        c[2] ^= i2
        c[3] ^= i3
        c[4] ^= i0
        c[5] ^= i1
        c[6] ^= i2
        c[7] ^= i3

        next(self)
        next(self)
        next(self)
        next(self)

    def __next__(self):
        c = self.c
        x = self.x
        b = self.b

        t = c[0] + 0x4D34D34D + b
        c[0] = t % WORD_SIZE
        t = c[1] + 0xD34D34D3 + t // WORD_SIZE
        c[1] = t % WORD_SIZE
        t = c[2] + 0x34D34D34 + t // WORD_SIZE
        c[2] = t % WORD_SIZE
        t = c[3] + 0x4D34D34D + t // WORD_SIZE
        c[3] = t % WORD_SIZE
        t = c[4] + 0xD34D34D3 + t // WORD_SIZE
        c[4] = t % WORD_SIZE
        t = c[5] + 0x34D34D34 + t // WORD_SIZE
        c[5] = t % WORD_SIZE
        t = c[6] + 0x4D34D34D + t // WORD_SIZE
        c[6] = t % WORD_SIZE
        t = c[7] + 0xD34D34D3 + t // WORD_SIZE
        c[7] = t % WORD_SIZE
        b = t // WORD_SIZE

        g = [non_linear_state_transition(x[j], c[j]) for j in range(8)]

        x[0] = (g[0] + rotate_by_16(g[7]) + rotate_by_16(g[6])) % WORD_SIZE
        x[1] = (g[1] + rotate_by_8(g[0]) + g[7]) % WORD_SIZE
        x[2] = (g[2] + rotate_by_16(g[1]) + rotate_by_16(g[0])) % WORD_SIZE
        x[3] = (g[3] + rotate_by_8(g[2]) + g[1]) % WORD_SIZE
        x[4] = (g[4] + rotate_by_16(g[3]) + rotate_by_16(g[2])) % WORD_SIZE
        x[5] = (g[5] + rotate_by_8(g[4]) + g[3]) % WORD_SIZE
        x[6] = (g[6] + rotate_by_16(g[5]) + rotate_by_16(g[4])) % WORD_SIZE
        x[7] = (g[7] + rotate_by_8(g[6]) + g[5]) % WORD_SIZE

        self.b = b
        return self

    def derive(self):
        x = self.x
        return ((x[0] & 0xFFFF) ^ (x[5] >> 16)) | \
                (((x[0] >> 16) ^ (x[3] & 0xFFFF)) << 16)| \
                (((x[2] & 0xFFFF) ^ (x[7] >> 16)) << 32)| \
                (((x[2] >> 16) ^ (x[5] & 0xFFFF)) << 48)| \
                (((x[4] & 0xFFFF) ^ (x[1] >> 16)) << 64)| \
                (((x[4] >> 16) ^ (x[7] & 0xFFFF)) << 80)| \
                (((x[6] & 0xFFFF) ^ (x[3] >> 16)) << 96)| \
                (((x[6] >> 16) ^ (x[1] & 0xFFFF)) << 112)

    def keystream(self, n):
        # Generate keystream
        result = ""
        buffer = self._buf
        remaining_bits = self._buf_bytes
        state_transition = self.__next__
        derive_function = self.derive

        for i in range(n):
            if not remaining_bits:
                remaining_bits = 16
                state_transition()
                buffer = derive_function()
            result += chr(buffer & 0xFF)
            remaining_bits -= 1
            buffer >>= 1

        self._buf = buffer
        self._buf_bytes = remaining_bits
        return result
    
    def save_keystream_to_file(self, filename, n):
        generated_keystream = self.keystream(n)

        with open(filename, 'wb') as file:
            binary_keystream = bytes(generated_keystream.encode())
            file.write(binary_keystream)

    def encrypt(self, data):
        result = ""
        buffer = self._buf
        remaining_bits = self._buf_bytes
        state_transition = self.__next__
        derive_function = self.derive

        for character in data:
            if not remaining_bits:
                remaining_bits = 16
                state_transition()
                buffer = derive_function()
            result += chr(ord(character) ^ (buffer & 0xFF))
            remaining_bits -= 1
            buffer >>= 1

        self._buf = buffer
        self._buf_bytes = remaining_bits
        return result

    def reset(self, iv=None):
        self.c = self.start_c[:]
        self.x = self.start_x[:]
        self.b = self.start_b
        self._buf = 0
        self._buf_bytes = 0
        if iv is not None:
            self.set_iv(iv)

    # Decrypt is same as encrypt
    decrypt = encrypt

In [16]:
# Test Rabbit encryption
plainText = "TruongThiQuyen 20203553"
key = "BTLLTMM"
iv = 2

In [17]:
key_hashed = hashlib.md5(key.encode()).hexdigest()
generated_keystream = Rabbit(key_hashed, iv).keystream(len(plainText))

In [18]:
# Save to file
Rabbit(key_hashed, iv).save_keystream_to_file("keystream.txt", len(plainText))

In [19]:
print("PlainText:\t\t", plainText)
print("IV:\t\t\t", iv)
print("Password:\t\t", key)
print("Key:\t\t\t", key_hashed)
print("Generated key stream:\t", binascii.hexlify(generated_keystream.encode()))

PlainText:		 TruongThiQuyen 20203553
IV:			 2
Password:		 BTLLTMM
Key:			 333a3f4415034c6ab9c444db4f4e26f5
Generated key stream:	 b'4f271309c284c382c3a170c2b85cc2aec3976bc2b5c39ac3adc387633118c28cc38663'


In [20]:
encrypted_message = Rabbit(key_hashed, iv).encrypt(plainText)
print("After Encrypted:\t", binascii.hexlify(encrypted_message.encode()))

decrypted_message = Rabbit(key_hashed, iv).decrypt(encrypted_message)
print("After Decrypted:\t", decrypted_message)

After Encrypted:	 b'1b556666c3aac2a5c2b518c3910dc39bc2ae0ec39bc3bac39fc3b751012bc2b9c3b350'
After Decrypted:	 TruongThiQuyen 20203553
